## The term pandonic exists for a reason

The term pandonic isn't as common as pythonic, but it exists. My sense is this term is used to distinguish between what is natural to python and what is natural to pandas. For example, there is temptation to treat a `DataFrame` like its a dictionary, and while you can often do that, it can get you in trouble with performance. 

Here is a very quick demonstration of this. 

(Don't be scared, I'm using `ticdat` here to grab some data, but I'll be talking about `pandas` in a second).

In [1]:
import ticdat.testing.ticdattestutils as tu
from ticdat import TicDatFactory
tdf = TicDatFactory(**tu.dietSchema())
dat = tdf.copy_tic_dat(tu.dietData())
df = tdf.copy_to_pandas(dat).nutritionQuantities

Ok, `df` is just a `DataFrame` with an index holding the primary key fields. See here.

In [2]:
df

qty
food      category        
hamburger calories   410.0
          protein     24.0
          fat         26.0
          sodium     730.0
chicken   calories   420.0
          protein     32.0
          fat         10.0
          sodium    1190.0
hot dog   calories   560.0
          protein     20.0
          fat         32.0
          sodium    1800.0
fries     calories   380.0
          protein      4.0
          fat         19.0
          sodium     270.0
macaroni  calories   320.0
          protein     12.0
          fat         10.0
          sodium     930.0
pizza     calories   320.0
          protein     15.0
          fat         12.0
          sodium     820.0
salad     calories   320.0
          protein     31.0
          fat         12.0
          sodium    1230.0
milk      calories   100.0
          protein      8.0
          fat          2.5
          sodium     125.0
ice cream calories   330.0
          protein      8.0
          fat         10.0
          sodium     180.0

Now, suppose you want to ask how much protein is in ice cream? You can do this.

In [3]:
 df.loc[("ice cream", "protein"), "qty"]

8.0

Hey it works, and it looks like reading from dictionary? So what's the problem? Well a real dictionary can be created from `df` like this.

In [4]:
dict_of_dicts = df.T.to_dict()

And then you read the same cell like this.

In [5]:
dict_of_dicts["ice cream", "protein"]["qty"]

8.0

Hey, tomato tuh-mah-toe, right? Umm, not really my friend. 

In [6]:
%timeit dict_of_dicts["ice cream", "protein"]["qty"]

123 ns ± 1.29 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [7]:
%timeit df.loc[("ice cream", "protein"), "qty"]

215 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now bear in mind, both lookups happen fast, but the true dictionary one is quite a bit faster. (Roughly 2000X faster). Depending on how often you're looking up specific rows based on specific values, that can be huge.

## ticdat digression

Now, if you really like dict-of-dicts, `ticdat` can set those up easily. 

In [8]:
dat.nutritionQuantities["ice cream", "protein"]["qty"]

8

So how fast is it?

In [9]:
%timeit dat.nutritionQuantities["ice cream", "protein"]["qty"]

921 ns ± 7.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


It's a lot closer to "true dictionary" performance than a `DataFrame`, but still almost 10X slower than a real dictionary. What's the deal? Well, `ticdat` overrides some dictionary performance to give more natural coding styles. You can either have a frozen data object, where all you can do is read data, or a `defaultdict` type object, where you can easily add rows and let default values populate data values that aren't set explicitly. That said, it is an outstanding [issue](https://github.com/opalytics/opalytics-ticdat/issues/215) to address this for `ticdat` and add an option so that tables can be rendered as true `dict_of_dicts` style objects if thats what's required. 